<a href="https://colab.research.google.com/github/pickledherring/NLP-group/blob/main/gloss_glove_sgns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import auth
from googleapiclient.discovery import build
from io import FileIO
from googleapiclient.http import MediaIoBaseDownload

import pandas as pd
import nltk
from sklearn.model_selection import train_test_split

import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error
from tensorflow.keras.losses import cosine_similarity

try:
  stop_words = nltk.corpus.stopwords.words("english")
except:
  nltk.download('stopwords')
  stop_words = nltk.corpus.stopwords.words("english")

import numpy as np
import torch
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
#  Get the files from google drive
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Get english train data file
file_id = '1m3Ax9Z8OHMU-7FqraKc-ddI3YQ7yY_Q6'  # file id on the Google Drive
downloaded = FileIO("en.trial.complete.json", 'w')
request = drive_service.files().get_media(fileId=file_id)
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  status, done = downloader.next_chunk()
  print("Download {}%.".format(int(status.progress() * 100)))


Download 100%.


In [20]:
en_df = pd.read_json("en.trial.complete.json")
en_df.head()

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
0,en.trial.1,beautiful,adjective,Pleasant ; clear .,"It 's beautiful outside , let 's go for a walk .",synonym/antonym-based,124908,706,0,0,"[1.393769145, 0.7516670227000001, -2.581333160...","[0.295645088, 0.098426342, 0.0463486575, 0.016...","[0.0800914839, -0.1875839084, -0.0411579385000..."
1,en.trial.2,cocktail,noun,A mixture of other substances or things .,a cocktail of illegal drugs,hypernym-based,4187,13245,1,0,"[2.0872907639, 0.2617726326, 0.668431639700000...","[0.3878918886, 0.1971583217, -0.44026631120000...","[-1.4771454334, -0.4742421806, 0.0847439319, -..."
2,en.trial.3,institutionalized,adjective,Having been established as an institution .,It is very difficult to get bureaucracies to a...,paraphrastic,961,35934,0,0,"[0.7893871069, -0.43510755900000003, 0.8553860...","[-0.0519028902, 0.2257766128, -0.1839749813, 0...","[-1.1030955315, -0.9046602845, 0.1503403783, -..."
3,en.trial.4,menial,noun,"A servant , especially a domestic servant .","The world was awake to the 2nd of May , but Ma...",hypernym-based,517,53267,1,1,"[0.1222261563, 0.1572209597, 0.5396134257, -0....","[-0.3667449057, -0.1431699395, -0.0671329796, ...","[-1.6584062576, -0.24498166140000002, 0.150174..."
4,en.trial.5,seek,verb,To try to find ; to look for ; to search for .,"Not long ago , it was difficult to produce pho...",paraphrastic,25195,3212,0,0,"[1.1894155741, 1.3668279648000001, -1.61634504...","[0.6137102246, 0.5464909673, -0.0161557049, 9....","[-0.5474479198000001, -0.0880863219, 0.0784259..."


In [48]:
en_df.dtypes

id            object
word          object
pos           object
gloss         object
example       object
type          object
counts         int64
f_rnk          int64
concrete       int64
polysemous     int64
sgns          object
char          object
electra       object
dtype: object

In [22]:
en_df[en_df.word.duplicated()]

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
66,en.trial.67,divorce,verb,To legally dissolve a marriage between two peo...,"A ship captain can marry couples , but can not...",paraphrastic,21171,3773,0,0,"[1.579477787, -1.7040250301, 2.4623465538, -0....","[-0.09191438560000001, -0.0440451615, -0.45280...","[0.0899147466, 0.0271891933, 0.023069024100000..."
188,en.trial.189,forthwith,adverb,Without delay ; immediately .,"Then Proclamation was made , that they that ha...",synonym/antonym-based,1179,31462,0,1,"[0.7841670513, 0.6406752467, -1.0482375622, -0...","[-8.645650000000001e-05, 0.161273554, 0.082766...","[-1.0183763504, -0.3864063621, 0.0048319194, -..."


we have words to disambiguate! forthwith looks to be the same with a different example though. <br>thoughts on disambiguation (feel free to add):<br><br> need to gather contex, but all I can think of is to train the NN to find the part of speech of the word and maybe type. I think we are only given the gloss and from that predict the embedding, so we (maybe) can't use the rest of it except in training. probably a good idea regardless of the disambiguation goal. <br><br> we could also retrain the model on incorrect words that are polysemous, like a boosting method. <br><br> lastly, disambiguation might not be necessary given the gloss and the vector might not be so similar. should look at the distance between these words in the embeddings. glosses look different, but I can see the top few elements of these two polysemous words are similar in all of the embeddings.

In [23]:
en_df[(en_df.word == "divorce") | (en_df.word == "forthwith")]

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
50,en.trial.51,divorce,verb,To separate something that was connected .,The radical group voted to divorce itself from...,hypernym-based,21171,3773,0,0,"[1.579477787, -1.7040250301, 2.4623465538, -0....","[-0.09191438560000001, -0.0440451615, -0.45280...","[-0.6520454288, -0.1928912848, 0.0298318155, -..."
66,en.trial.67,divorce,verb,To legally dissolve a marriage between two peo...,"A ship captain can marry couples , but can not...",paraphrastic,21171,3773,0,0,"[1.579477787, -1.7040250301, 2.4623465538, -0....","[-0.09191438560000001, -0.0440451615, -0.45280...","[0.0899147466, 0.0271891933, 0.023069024100000..."
160,en.trial.161,forthwith,adverb,Without delay ; immediately .,"Let ther be Light , said God , and forthwith L...",synonym/antonym-based,1179,31462,0,1,"[0.7841670513, 0.6406752467, -1.0482375622, -0...","[-8.645650000000001e-05, 0.161273554, 0.082766...","[-0.9691627026, -0.20675842460000002, 0.184671..."
188,en.trial.189,forthwith,adverb,Without delay ; immediately .,"Then Proclamation was made , that they that ha...",synonym/antonym-based,1179,31462,0,1,"[0.7841670513, 0.6406752467, -1.0482375622, -0...","[-8.645650000000001e-05, 0.161273554, 0.082766...","[-1.0183763504, -0.3864063621, 0.0048319194, -..."


clean glosses of punctuation, stopwords, duplicates, turn into lists.

In [24]:
def clean(gloss):
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  cleaned = tokenizer.tokenize(gloss)
  cleaned = list(set([word.lower() for word in cleaned]))
  cleaned = [word for word in cleaned if not word in stop_words]
  return cleaned

gloss_lists = en_df.gloss.apply(clean)
gloss_lists

0                                      [clear, pleasant]
1                          [substances, things, mixture]
2                             [established, institution]
3                        [domestic, especially, servant]
4                              [look, try, search, find]
                             ...                        
195                        [animal, color, plant, cells]
196                                [seeming, appearance]
197       [vehicle, permission, proceed, person, travel]
198                      [sitting, around, moving, much]
199    [word, phrase, heraldic, achievement, part, se...
Name: gloss, Length: 200, dtype: object

In [25]:
# list of all context words
context_voc = []
for i in range(len(gloss_lists)):
  for j in range(len(gloss_lists[i])):
    if not gloss_lists[i][j] in context_voc:
      context_voc.append(gloss_lists[i][j])

based on an SGNS following this guide: https://medium.com/towards-datascience/word2vec-negative-sampling-made-easy-7a1a647e07a4


In [26]:
# true context words for each defined word (center word)
trues = []
for i in range(len(gloss_lists)):
  for j in range(len(gloss_lists[i])):
    index = context_voc.index(gloss_lists[i][j])
    # append index of center in gloss_lists, index of context in context_voc, and 1 for true
    trues.append([i, index, 1])

In [27]:
trues[:5]

[[0, 0, 1], [0, 1, 1], [1, 2, 1], [1, 3, 1], [1, 4, 1]]

this could be improved by weighting the selection by the count of the word raised to 3/4 over the sum of the counts of all words raised to 3/4.

we can also try resampling with each epoch

In [28]:
falses = []
# create 3 randomly sampled context words for each true context word
# these may be true, but probably not. we'll label them false
for i in range(len(trues)):
  for j in range(3):
    center_index = trues[i][0]
    context_index = random.sample(range(len(context_voc)), 1)[0]
    falses.append([center_index, context_index, 0])

In [29]:
def gen_onehot():
  # combine and shuffle trues and falses
  together = np.concatenate((np.array(trues), np.array(falses)))
  np.random.shuffle(together)
  targets = torch.Tensor(together).long()

  # matrices to one hot encode middle words and target words
  middle_tensor = torch.zeros(targets.shape[0], gloss_lists.shape[0])
  context_tensor = torch.zeros(targets.shape[0], len(context_voc))

  for i in range(middle_tensor.shape[0]):
    middle_tensor[i, targets[i, 0]] = 1
    context_tensor[i, targets[i, 1]] = 1

  labels = targets[:, 2].float()
  return (middle_tensor, context_tensor, labels)

build the model itself below. not sure why bias needs to be false, should try as true.

this would be more elegant as a class or at least a function combined with the forward (dot product production) part included.

In [30]:
# fully connected middle layers for middle and target words
mid_fc = torch.nn.Linear(gloss_lists.shape[0], 256, bias = False)
con_fc = torch.nn.Linear(len(context_voc), 256, bias = False)
torch.nn.init.xavier_uniform_(mid_fc.weight)
torch.nn.init.xavier_uniform_(con_fc.weight)
sig = torch.nn.Sigmoid()
params = list(mid_fc.parameters()) + list(con_fc.parameters())
optim = torch.optim.Adam(params, lr = .001)
loss_fn = torch.nn.BCELoss()

In [ ]:
epochs = 100
mid_hot, con_hot, labels = gen_onehot()

for i in range(epochs):
  trans_mid = mid_fc(torch.Tensor(mid_hot))
  trans_con = con_fc(torch.Tensor(con_hot))
  dot_mat = torch.zeros(mid_hot.shape[0], 1)
  # for each row dot a center embedding by a target embedding
  for j in range(len(trans_mid)):
    dot_mat[j, :] = trans_mid[j, :] @ trans_con[j, :]
  # sigmoid transformation, then compute the gradient and backwards propagate
  prob_mat = sig(dot_mat)
  prob_mat.requires_grad = True
  optim.zero_grad()
  loss = loss_fn(prob_mat, torch.Tensor(labels).view(prob_mat.shape[0], 1))
  loss.backward()
  optim.step()

  # print loss every 10 epochs
  if i % 10 == 0:
    print(loss.data)
  mid_hot, con_hot, labels = gen_onehot()

In [32]:

print(loss.data)

tensor(0.6936)


In [33]:
embeddings = mid_fc.weight.t().detach().numpy()
X_train, X_test, y_train, y_test = train_test_split(embeddings, en_df.word)

In [47]:
#print(X_test.shape, "\n", y_test.shape)
print("xtrain: ", X_train, "\nytrain: ", y_train)

xtrain:  [[-0.08244613  0.05170777 -0.10481814 ... -0.05728389 -0.02333343
  -0.0198084 ]
 [-0.06401545  0.03203784  0.10222124 ... -0.00624143 -0.00557776
  -0.01500468]
 [-0.08806799  0.03453511  0.11092366 ...  0.05850323  0.03197394
  -0.02470952]
 ...
 [ 0.10356567 -0.00278639  0.00043946 ... -0.10813225 -0.04634301
  -0.06341793]
 [-0.01858965  0.0857036   0.0029063  ... -0.09698204  0.05720955
   0.07247003]
 [ 0.00333329 -0.0653206   0.05320363 ... -0.08313783 -0.03336279
  -0.0164146 ]] 
ytrain:  119      previous
10        scraggy
25           lull
123      cover-up
124       version
          ...    
116         grape
199         motto
102       caravan
54     seriocomic
76           ride
Name: word, Length: 150, dtype: object


In [ ]:
X_train = np.asarray( X_train, dtype = np.float32 )
y_train = np.asarray( y_train, dtype = np.float32 )
X_test = np.asarray( X_test, dtype = np.float32 )
y_test = np.asarray( y_test, dtype = np.float32 )

NN Model

In [39]:
inputs = tf.keras.Input(shape=(256,), dtype="float64")

x = layers.Dropout(0.1)(inputs)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)

predictions = layers.Dense(256, activation="tanh", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


Training Model

In [44]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(X_train, y_train, epochs=epochs)

Epoch 1/3


UnimplementedError: ignored

Testing

Cosine Similarity

In [ ]:
#cosine_loss(y_true, y_pred).numpy()

Mean Squared Error

In [ ]:
#mean_squared_error(y_true, y_pred)